# 🍈 **요새 꽂힌 그 노래와 유사한 곡이 궁금해!** 🎶
* 유사곡을 찾고 싶은 노래의 제목을 입력하세요.
* 추천곡의 멜론 링크를 제공합니다 !

### 📂 **추천에 필요한 파일 가져오기**

In [ ]:
import pickle
import pandas as pd
import re
import random
from urllib import parse

In [ ]:
# 유사곡이 저장된 dictionary 불러오기

song_meta = pd.read_pickle('/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/song_meta_nov.pkl')

with open('/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/nov/sim_songs_epoch30_cos_dict.pkl', 'rb') as f: # sid로 바뀌어있음 !
      sim_songs = pickle.load(f)

In [219]:
song_meta = song_meta.loc[list(sim_songs.keys())]
song_meta_subset = song_meta[['album_name','song_name','artist_name_basket']]
song_meta_subset.rename(columns = {'album_name':'앨범 제목은 !','song_name':'노래 제목은 !','artist_name_basket':'가수는 누구인가 !'}, inplace = True)
sid_to_meta = song_meta.set_index('id')[['album_name','song_name','artist_name_basket']].T.to_dict()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# 👩‍💻 **Trie와 RecSongs class를 정의합니다.**

## 🌳 **1. 빠른 노래 제목 검색을 위한 DB를 구성합니다.**
* Trie를 통해 빠른 탐색을 합니다.
* 제목이 같은 노래가 있다면 list에 담아 return합니다.

In [275]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = {}
        self.sid = [] # terminal flag 대신 sid의 유무로 판단

class Trie:
    def __init__(self, sid_list, song_list):
        self.head = Node(None)

        print("********* DB 구성중입니다 *********")
        for sid, title in zip(sid_list, song_list):
            self.insert(sid, title)
        print("************ 입력 완료 ************")
        
    def get_plain(self, query):
        plain = re.findall('\w', query.split("(")[0])

        if plain is None:
            return None
        
        return "".join(plain).lower()

    def insert(self, sid, query):
        plain_query = self.get_plain(query)

        if plain_query is None:
            self.head.children[query] = Node(query)
            self.head.children[query].sid.append(sid)
            return

        curr_node = self.head

        for q in plain_query:
            if curr_node.children.get(q) is None:
                curr_node.children[q] = Node(q)
            curr_node = curr_node.children[q]
        curr_node.sid.append(sid)

    def search(self, query):
        plain_query = self.get_plain(query)

        if plain_query is None:
            candidate = self.head.children.get(query)
            return candidate.sid if candidate is not None else False

        curr_node = self.head

        for q in plain_query:
            curr_node = curr_node.children.get(q)
            if curr_node is None:
                return False
        
        if len(curr_node.sid) == 0:
            return False

        return curr_node.sid

In [276]:
trie = Trie(song_meta.id, song_meta.song_name)

********* DB 구성중입니다 *********
************ 입력 완료 ************


In [277]:
song_meta.query(f'id in {trie.search("금요일에만나요")}')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
549178,"[GN0105, GN0101]",20131220,Modern Times - Epilogue,2222587,[261143],금요일에 만나요 (Feat. 장이정 Of HISTORY),[GN0100],[아이유],549178,"[겨울, 출근길, 월요일, 월요병, 지하철, 드라이브, 스트레스, 버스, 기분전환,..."


In [278]:
song_meta.query(f'id in {trie.search("봄~사랑~벚꽃 말고 ~")}')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
650494,"[GN0105, GN0101]",20140408,봄 사랑 벚꽃 말고,2248659,"[767374, 261143]",봄 사랑 벚꽃 말고,[GN0100],"[HIGH4 (하이포), 아이유]",650494,"[매장, 여름, 카페, 봄, 휴식, 힐링, 잔잔한, 팝, 설렘, 사랑, 힙합, 기분..."
18039,[GN0101],20140408,봄 사랑 벚꽃 말고,2248659,"[767374, 261143]",봄 사랑 벚꽃 말고 (Inst.),[GN0100],"[HIGH4 (하이포), 아이유]",18039,"[설렘, 사랑, 힐링, 휴식, 기분전환, 힙합, 랩, 까페, 잔잔한, 아이유, 그루..."


## 🧀 **2. top k개의 유사곡을 추천합니다.**
* `topk` 변수를 통해 추천 곡 수를 조절합니다.
* 제목이 같은 곡이 여러개라면 사용자가 선택하도록 합니다.
* 없는 곡을 입력한 경우 랜덤으로 한 곡을 추천합니다.
* topk개의 곡 중 랜덤으로 하나의 곡의 링크를 제공합니다.

In [363]:
class RecSongs:
    def __init__(self, database , topk = 10):
        self.index_emoji = ['💛','💚','💙','💜','🧡']
        self.k = topk
        self.sim_songs = {sid:sim[:topk] for sid,sim in sim_songs.items()} # topk개만큼 추천한다.
        self.sid_list = list(self.sim_songs.keys())
        self.db = database # trie object를 DB로 이용한다.

    def get_url(self, query):
        url_encoding = parse.quote(query)
        return f'https://www.melon.com/search/song/index.htm?q={url_encoding}'

    def random_recommendation(self, candidates = False):
        if not candidates:
            random_sid = random.choice(self.sid_list)
        else:
            random_sid = random.choice(candidates)
            
        return self.get_url(sid_to_meta[random_sid]['song_name'])

    def recommend_sim_songs(self):
        query = input("👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  ")
        results = self.db.search(query)

        if not results:
            print(f"\n😥 {query}...? 아직 잘 모르는 노래에요 ㅠ_ㅠ 꼭 한 번 들어볼게요")
            
            print("\n🎁 그냥 가기 아쉬우니 아무 노래나 추천 ! 멜론에서 들어보아요 ! > ",self.random_recommendation() )
            return
        
        index = 0

        if len(results) >= 2:
            for i,sid in enumerate(results):
                album, song, artist = sid_to_meta[sid].values()
                print(f'\n\t {self.index_emoji[i%5]} {i+1} < {" , ".join(artist)} > 의 앨범 < {album} > 에 담긴 노래 << {song} >>')

            index = int(input(f"\n💁‍♂ {1}부터 {len(results)}까지 중에서 몇 번째 노래인지 골라주세요 ! : "))
            index -= 1

        sid = results[index]
        album, song, artist = sid_to_meta[sid].values()
        candidates = self.sim_songs[sid]

        recommendation = song_meta_subset.loc[candidates].reset_index(drop=True)

        print(f"\n🎵 {' , '.join(artist)}의 {song} 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈\n")
        display(recommendation)

        print("\n🎁 랜덤 추천 ! 멜론에서 들어보아요 ! > ",self.random_recommendation(candidates))

In [364]:
rec_songs = RecSongs(trie,8)

#🙋 **유사곡 top10을 추천해보자 !**

## 🌸 **아이유의 <봄 사랑 벚꽃말고>와 유사한 곡을 추천해줘 !**
* topk의 추천곡 중 랜덤으로 한 곡의 멜론 링크를 제공합니다.

In [365]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  봄 ~ 사랑 ~ 벚꽃 말고 ~

	 💛 1 < HIGH4 (하이포) , 아이유 > 의 앨범 < 봄 사랑 벚꽃 말고 > 에 담긴 노래 << 봄 사랑 벚꽃 말고 >>

	 💚 2 < HIGH4 (하이포) , 아이유 > 의 앨범 < 봄 사랑 벚꽃 말고 > 에 담긴 노래 << 봄 사랑 벚꽃 말고 (Inst.) >>

💁‍♂ 1부터 2까지 중에서 몇 번째 노래인지 골라주세요 ! : 1

🎵 HIGH4 (하이포) , 아이유의 봄 사랑 벚꽃 말고 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,냄새를 보는 소녀 OST Part.2,우연히 봄,"[로꼬, 유주 (여자친구)]"
1,봄 타나봐 (BOMTANABA),봄 타나봐 (BOMTANABA),[서인국]
2,봄인가 봐 (Spring Love) - SM STATION,봄인가 봐 (Spring Love),"[에릭남 (Eric Nam), 웬디 (WENDY)]"
3,썸,썸 (Feat. 릴보이 Of 긱스),"[소유 (SOYOU), 정기고]"
4,Modern Times - Epilogue,금요일에 만나요 (Feat. 장이정 Of HISTORY),[아이유]
5,버스커 버스커 1집,벚꽃 엔딩,[버스커 버스커]
6,The 3rd Album Part.2 `Love Blossom (러브블러썸)`,Love Blossom (러브블러썸),[케이윌]
7,PLAY,Give Love,[AKMU (악동뮤지션)]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EC%9A%B0%EC%97%B0%ED%9E%88%20%EB%B4%84


## 👻 **<죠르디 귀여워 쏭> 과 유사한 곡을 추천해줘 !**
* DB에 없는 노래를 입력한 경우 랜덤으로 노래 한 곡을 추천합니다.

In [366]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  죠르디 귀여워 쏭

😥 죠르디 귀여워 쏭...? 아직 잘 모르는 노래에요 ㅠ_ㅠ 꼭 한 번 들어볼게요

🎁 그냥 가기 아쉬우니 아무 노래나 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EB%84%88%EC%9D%98%20%EA%B3%BC%EA%B1%B0%EA%B0%80%20%EB%90%A0%20%EC%9A%A9%EA%B8%B0


## 🌷 **오마이걸의 <비밀정원>과 유사한 곡을 추천해줘 !**

In [367]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  비밀 정원

	 💛 1 < 오마이걸 > 의 앨범 < 비밀정원 > 에 담긴 노래 << 비밀정원 >>

	 💚 2 < 러블리즈 > 의 앨범 < Lovelyz 3rd Mini Album (Fall in Lovelyz) > 에 담긴 노래 << 비밀정원 >>

	 💙 3 < 그레이프 티 (Grape T) > 의 앨범 < Blossom > 에 담긴 노래 << 비밀 정원 >>

	 💜 4 < 러블리즈 > 의 앨범 < Muse on Music > 에 담긴 노래 << 비밀정원 >>

💁‍♂ 1부터 4까지 중에서 몇 번째 노래인지 골라주세요 ! : 1

🎵 오마이걸의 비밀정원 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,OH MY GIRL 6th Mini Album (REMEMBER ME),불꽃놀이 (Remember Me),[오마이걸]
1,What is Love?,What is Love?,[TWICE (트와이스)]
2,여자친구 The 6th Mini Album &#39;Time for the moon...,밤 (Time for the moon night),[여자친구 (GFRIEND)]
3,Perfect Velvet - The 2nd Album,피카부 (Peek-A-Boo),[Red Velvet (레드벨벳)]
4,From. 우주소녀,너에게 닿기를 (I Wish),[우주소녀]
5,"Lovelyz 2nd Album Repackage `지금, 우리`","지금, 우리",[러블리즈]
6,Merry & Happy,Heart Shaker,[TWICE (트와이스)]
7,GREAT!,뿜뿜,[모모랜드 (MOMOLAND)]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EB%B0%A4%20%28Time%20for%20the%20moon%20night%29


## 🎤 **장범준의 <노래방에서>와 유사한 곡을 추천해줘!**

In [368]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  노래방에서~

	 💛 1 < 반하나 > 의 앨범 < 노래방에서 > 에 담긴 노래 << 노래방에서 >>

	 💚 2 < 장범준 > 의 앨범 < 장범준 3집 > 에 담긴 노래 << 노래방에서 >>

	 💙 3 < 서은광 , 앤씨아 > 의 앨범 < 노래방에서 > 에 담긴 노래 << 노래방에서 >>

	 💜 4 < 서은광 , 앤씨아 > 의 앨범 < 노래방에서 > 에 담긴 노래 << 노래방에서 (Inst.) >>

💁‍♂ 1부터 4까지 중에서 몇 번째 노래인지 골라주세요 ! : 2

🎵 장범준의 노래방에서 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,장범준 3집,당신과는 천천히,[장범준]
1,전설,주저하는 연인들을 위해,[잔나비]
2,진심이 담긴 노래 (True Song),진심이 담긴 노래 (True Song),[케이시 (Kassy)]
3,FLY HIGH PROJECT #2 `옥탑방`,옥탑방 (Rooftop),[엔플라잉 (N.Flying)]
4,사춘기집Ⅰ 꽃기운,"나만, 봄",[볼빨간사춘기]
5,그때가 좋았어,그때가 좋았어,[케이시 (Kassy)]
6,Spring,봄 (feat. 산다라박),[박봄]
7,사춘기집Ⅰ 꽃기운,별 보러 갈래?,[볼빨간사춘기]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EB%82%98%EB%A7%8C%2C%20%EB%B4%84


## 💚 **백예린의 < Square >와 유사한 곡을 추천해줘!**

In [369]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  square

	 💛 1 < 백예린 > 의 앨범 < Every letter I sent you. > 에 담긴 노래 << Square (2017) >>

	 💚 2 < 플라잉비83 > 의 앨범 < One Two Zero > 에 담긴 노래 << Square >>

	 💙 3 < Ryan Bader > 의 앨범 < Tech House Lady Lover > 에 담긴 노래 << Square >>

💁‍♂ 1부터 3까지 중에서 몇 번째 노래인지 골라주세요 ! : 1

🎵 백예린의 Square (2017) 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,Love poem,Blueming,[아이유]
1,Our love is great,야간비행 (魔女の花),[백예린]
2,From Midnight To Sunrise,Alone,[Crush]
3,사랑의 불시착 OST Part 4,"다시 난, 여기",[백예린]
4,From Midnight To Sunrise,With You,[Crush]
5,Red Moon : Happiness,마음이 말하는 행복 (Happiness) (Feat.이라온),[마크툽 (MAKTUB)]
6,Loveship,Loveship,"[폴킴, 청하]"
7,혼자,혼자,"[다이나믹 듀오, 첸 (CHEN)]"



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EC%95%BC%EA%B0%84%EB%B9%84%ED%96%89%20%28%E9%AD%94%E5%A5%B3%E3%81%AE%E8%8A%B1%29


## 🎸 **Pink Sweat$의 < Honesty >와 유사한 곡을 추천해줘 !**

In [371]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  honesty

	 💛 1 < Beyonce > 의 앨범 < I Am... Sasha Fierce (Platinum Edition) > 에 담긴 노래 << Honesty >>

	 💚 2 < Billy Joel > 의 앨범 < Piano Man - The Very Best Of Billy Joel > 에 담긴 노래 << Honesty >>

	 💙 3 < Billy Joel > 의 앨범 < 이달의 팝 : 365일, 팝 음악을 만나다 > 에 담긴 노래 << Honesty >>

	 💜 4 < Landser > 의 앨범 < Jazzin` Cafe > 에 담긴 노래 << Honesty >>

	 🧡 5 < Pink Sweat$ > 의 앨범 < Volume 1 EP > 에 담긴 노래 << Honesty >>

	 💛 6 < Pink Sweat$ > 의 앨범 < Honesty (Remix) [feat. Jessie Reyez] > 에 담긴 노래 << Honesty (Remix) [feat. Jessie Reyez] >>

	 💚 7 < Billy Joel > 의 앨범 < She`s Got A Way : Love Songs > 에 담긴 노래 << Honesty >>

	 💙 8 < 신화 > 의 앨범 < Perfect Man > 에 담긴 노래 << Honesty >>

	 💜 9 < Justin Park > 의 앨범 < Honesty (Pink Sweat$ Cover) > 에 담긴 노래 << Honesty (Pink Sweat$ Cover) >>

	 🧡 10 < Billy Joel > 의 앨범 < 52nd Street > 에 담긴 노래 << Honesty >>

	 💛 11 < Billy Joel > 의 앨범 < Billy Joel : She`s Always A Woman : The Love Songs > 에 담긴 노래 << Honesty >>

	 💚 12 < European Jazz Trio > 의 앨범 

,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,Selfless,EX,[Kiana Lede]
1,Soul Searching,I.F.L.Y.,[Bazzi]
2,right here,right here,[keshi]
3,Painkiller,Painkiller,[Ruel]
4,MINT,Tango,[Abir]
5,2 soon,2 soon,[keshi]
6,H.E.R.,Best Part (Feat. Daniel Caesar),[H.E.R.]
7,"summer,",comethru,[Jeremy Zucker]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=Painkiller


## **검정치마의 <나랑 아니면>과 유사한 곡을 추천해줘!**

In [372]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  나랑 아니면

🎵 검정치마의 나랑 아니면 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,EVERYTHING,EVERYTHING,[검정치마]
1,She,She (Hidden Track No.V 1월 선정곡),[잔나비]
2,MONKEY HOTEL,뜨거운 여름밤은 가고 남은 건 볼품없지만,[잔나비]
3,1,No One Told Me Why,[알레프 (ALEPH)]
4,Note#4,Alone,"[Collective Arts (콜렉티브아츠), 김현창]"
5,DEAR : X,선을 그어 주던가,[1415]
6,TEAM BABY,한시 오분 (1:05),[검정치마]
7,APARTMENT,Home Sweet Home,[카더가든]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=EVERYTHING


## **죠지의 < Boat >와 유사한 곡을 추천해줘 !**

In [373]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  boat

🎵 죠지의 Boat 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,Your Dog Loves You,Your Dog Loves You (Feat. Crush),[Colde (콜드)]
1,RIDE,RIDE (Feat. THAMA),[SOLE (쏠)]
2,cassette,let`s go picnic,[죠지]
3,KARMA,진심,[jeebanoff (지바노프)]
4,Sun And Moon,Make Up (Feat. Crush),[샘김 (Sam Kim)]
5,HANGANG,한강 (HANGANG),[Hoody (후디)]
6,Sunshine,Sunshine (Feat. Crush),[Hoody (후디)]
7,boy.,춤,[offonoff (오프온오프)]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=Make%20Up%20%28Feat.%20Crush%29


In [382]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  바라봐줘요

🎵 죠지의 바라봐줘요 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,너는 어떻게 (Feat. 백예린),너는 어떻게 (Feat. 백예린),[구원찬]
1,너라는 세상,너라는 세상,[스탠딩 에그]
2,"안녕, 나의 우주",우주선,[정승환]
3,봄이 오는 날에,봄이 오는 날에,[김민석]
4,Good Night,Good Night,[김민석]
5,전설,주저하는 연인들을 위해,[잔나비]
6,Polaris,우리 오늘 만날까,[이민혁]
7,Illusion,"쏟아져, 밤",[40]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EB%84%88%EB%9D%BC%EB%8A%94%20%EC%84%B8%EC%83%81


## **colde의 <없어도 돼>와 유사한 곡을 추천해줘 !**

In [375]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  없어도돼

🎵 Colde (콜드)의 없어도 돼 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,하다진,야..난,[리햅 (Rheehab)]
1,Can I Love ?,"Can I Love ? (Feat. 유라(youra), Meego)",[Cosmic Boy]
2,We (OUI),We (OUI) (Feat. sogumm),[jeebanoff (지바노프)]
3,Dingo X 죠지,camping everywhere,[죠지]
4,Is You Down,Is You Down (Feat. Minshik & Harms),[Jimmy Brown (지미 브라운)]
5,Wave,선 (Feat. 선우정아),[Colde (콜드)]
6,X,X,[D.SANGUH (딥상어)]
7,Love part 1,향,[Colde (콜드)]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=%EC%84%A0%20%28Feat.%20%EC%84%A0%EC%9A%B0%EC%A0%95%EC%95%84%29


## **DPR live의 <jasmine>과 유사한 곡 추천해줘 !**

In [386]:
rec_songs.recommend_sim_songs()

👩‍🏫 유사곡을 추천받고 싶은 노래 제목을 알려주세요 ! :  jasmine

	 💛 1 < DPR LIVE > 의 앨범 < Jasmine > 에 담긴 노래 << Jasmine >>

	 💚 2 < DJ soulscape > 의 앨범 < Lovers: > 에 담긴 노래 << Jasmine >>

	 💙 3 < Common Souls > 의 앨범 < Jasmine > 에 담긴 노래 << Jasmine >>

	 💜 4 < DPR LIVE > 의 앨범 < Her > 에 담긴 노래 << Jasmine >>

	 🧡 5 < 피아 (PIA) > 의 앨범 < Waterfalls > 에 담긴 노래 << Jasmine >>

	 💛 6 < Eevee > 의 앨범 < Unexpected > 에 담긴 노래 << Jasmine >>

	 💚 7 < 데이드림 > 의 앨범 < The Best Of Daydream - Sentimental Dream > 에 담긴 노래 << Jasmine >>

	 💙 8 < 레비트 > 의 앨범 < Treble Clef Of Happiness > 에 담긴 노래 << Jasmine >>

💁‍♂ 1부터 8까지 중에서 몇 번째 노래인지 골라주세요 ! : 1

🎵 DPR LIVE의 Jasmine 저도 좋아해요 ! 여기 유사한 곡이에요 ! 🍈



,앨범 제목은 !,노래 제목은 !,가수는 누구인가 !
0,Her,Martini Blue,[DPR LIVE]
1,Her,Text Me,[DPR LIVE]
2,boy.,gold (Feat. Dean),[offonoff (오프온오프)]
3,Coming To You Live,Laputa (Feat. Crush),[DPR LIVE]
4,Action!,Action! (Feat. GRAY),[DPR LIVE]
5,H.A.L.F (Have.A.Little.Fun),party (SHUT DOWN) (Feat. 크러쉬),[식케이 (Sik-K)]
6,EVERYWHERE,"Sunday (Feat. 헤이즈, 박재범)",[그루비룸 (GroovyRoom)]
7,Her,Is You Down,[DPR LIVE]



🎁 랜덤 추천 ! 멜론에서 들어보아요 ! >  https://www.melon.com/search/song/index.htm?q=party%20%28SHUT%20DOWN%29%20%28Feat.%20%ED%81%AC%EB%9F%AC%EC%89%AC%29
